Background: https://app.asana.com/0/0/1200719394714319/f

Previous fixes only manually re-assigned MAZ to parcels with MAZ=999999 and hh>0 in 2015 or 2050. This script identifies MAZ999999 parcels with (hh>0 in 2035) or (emp>0 in 2015) or (emp>0 in 2035) or (emp>0 in 2050), and reassign them based on visual inspection in GIS.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os

#### Read parcel/MAZ lookup files and BAUS run output

In [2]:
# p10 parcel ID - TM2 maz lookup (created after the previous fix on Aug 9, 2021)
p_maz_lookup = pd.read_csv('M:\\Data\\GIS layers\\p10_TM2_maz\\p10_maz\\p10_maz_lookup_compare_20210809.csv')
print('Read {} rows of parcel_id/maz lookup table with {} unique parcels and {} unique MAZs'.format(
    p_maz_lookup.shape[0],
    len(p_maz_lookup.PARCEL_ID.unique()),
    len(p_maz_lookup.maz_new.unique())))

print(p_maz_lookup.dtypes)
display(p_maz_lookup.head())

Read 1956208 rows of parcel_id/maz lookup table with 1956208 unique parcels and 39129 unique MAZs
PARCEL_ID       int64
maz_old         int64
maz_new       float64
maz_source     object
compare        object
dtype: object


,PARCEL_ID,maz_old,maz_new,maz_source,compare
0,229116,310596,324291.0,spatial join,diff
1,244166,331415,331415.0,spatial join,same
2,202378,310099,323229.0,spatial join,diff
3,2004420,710778,718260.0,spatial join,diff
4,340332,318182,318182.0,spatial join,same


In [3]:
# Read FBP 2015, 2035, 2050 UrbanSim output

# Final Blueprint run (run 182)
baus_output_folder = 'C:\\Users\\{}\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Final Blueprint runs\\Final Blueprint (s24)\\BAUS v2.25 - FINAL VERSION'.format(os.getenv('USERNAME'))

fbp_2015 = pd.read_csv(os.path.join(baus_output_folder, 'run182_parcel_data_2015.csv'),
                       usecols = ['parcel_id',
                                  'hhq1','hhq2','hhq3','hhq4','tothh',
                                  'AGREMPN', 'MWTEMPN', 'RETEMPN', 'FPSEMPN', 'HEREMPN', 'OTHEMPN', 'totemp'])
print('Read {} rows of FBP 2015 UrbanSim output data'.format(fbp_2015.shape[0]))
fbp_2015.rename(columns={'parcel_id': 'PARCEL_ID'},inplace=True)

fbp_2035 = pd.read_csv(os.path.join(baus_output_folder, 'run182_parcel_data_2035_UBI.csv'),
                       usecols = ['parcel_id',
                                  'hhq1', 'hhq2', 'hhq3', 'hhq4', 'tothh',
                                  'AGREMPN', 'MWTEMPN', 'RETEMPN', 'FPSEMPN', 'HEREMPN', 'OTHEMPN', 'totemp'])
print('Read {} rows of FBP 2035 UrbanSim output data'.format(fbp_2035.shape[0]))
fbp_2035.rename(columns={'parcel_id': 'PARCEL_ID'},inplace=True)

fbp_2050 = pd.read_csv(os.path.join(baus_output_folder, 'run182_parcel_data_2050_UBI.csv'),
                       usecols = ['parcel_id',
                                 'hhq1', 'hhq2', 'hhq3', 'hhq4', 'tothh',
                                 'AGREMPN', 'MWTEMPN', 'RETEMPN', 'FPSEMPN', 'HEREMPN', 'OTHEMPN', 'totemp'])
print('Read {} rows of FBP 2050 UrbanSim output data'.format(fbp_2050.shape[0]))
fbp_2050.rename(columns={'parcel_id': 'PARCEL_ID'},inplace=True)

# fbp_2015_file = 'C:\\Users\\{}\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Final Blueprint runs\\Final Blueprint (s24)\\BAUS v2.25 - FINAL VERSION\\'.format(os.getenv('USERNAME'))
# fbp_2015 = pd.read_csv(fbp_2015_file, usecols = ['parcel_id', 'hhq1','hhq2','hhq3','hhq4','tothh'])

print(fbp_2015.dtypes)
display(fbp_2015.head())

Read 1956212 rows of FBP 2015 UrbanSim output data
Read 1956212 rows of FBP 2035 UrbanSim output data
Read 1956212 rows of FBP 2050 UrbanSim output data
PARCEL_ID      int64
hhq1         float64
hhq2         float64
hhq3         float64
hhq4         float64
tothh        float64
AGREMPN      float64
MWTEMPN      float64
RETEMPN      float64
FPSEMPN      float64
HEREMPN      float64
OTHEMPN      float64
totemp       float64
dtype: object


,PARCEL_ID,hhq1,hhq2,hhq3,hhq4,tothh,AGREMPN,MWTEMPN,RETEMPN,FPSEMPN,HEREMPN,OTHEMPN,totemp
0,229116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,244166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202378,2.0,7.0,7.0,14.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,340332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# merge BAUS output with parcel/MAZ lookup
fbp_2015_maz = fbp_2015.merge(p_maz_lookup, on='PARCEL_ID', how='left')
display(fbp_2015_maz.head())

fbp_2035_maz = fbp_2035.merge(p_maz_lookup, on='PARCEL_ID', how='left')
display(fbp_2035_maz.head())

fbp_2050_maz = fbp_2050.merge(p_maz_lookup, on='PARCEL_ID', how='left')
display(fbp_2050_maz.head())

,PARCEL_ID,hhq1,hhq2,hhq3,hhq4,tothh,AGREMPN,MWTEMPN,RETEMPN,FPSEMPN,HEREMPN,OTHEMPN,totemp,maz_old,maz_new,maz_source,compare
0,229116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,310596.0,324291.0,spatial join,diff
1,244166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,331415.0,331415.0,spatial join,same
2,202378,2.0,7.0,7.0,14.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,310099.0,323229.0,spatial join,diff
3,2004420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,710778.0,718260.0,spatial join,diff
4,340332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,318182.0,318182.0,spatial join,same


,PARCEL_ID,hhq1,hhq2,hhq3,hhq4,tothh,AGREMPN,MWTEMPN,RETEMPN,FPSEMPN,HEREMPN,OTHEMPN,totemp,maz_old,maz_new,maz_source,compare
0,229116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,310596.0,324291.0,spatial join,diff
1,244166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,331415.0,331415.0,spatial join,same
2,202378,4.0,6.0,6.0,15.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,310099.0,323229.0,spatial join,diff
3,2004420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,710778.0,718260.0,spatial join,diff
4,340332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,318182.0,318182.0,spatial join,same


,PARCEL_ID,hhq1,hhq2,hhq3,hhq4,tothh,AGREMPN,MWTEMPN,RETEMPN,FPSEMPN,HEREMPN,OTHEMPN,totemp,maz_old,maz_new,maz_source,compare
0,229116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,310596.0,324291.0,spatial join,diff
1,244166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,331415.0,331415.0,spatial join,same
2,202378,2.0,5.0,10.0,16.0,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,310099.0,323229.0,spatial join,diff
3,2004420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,710778.0,718260.0,spatial join,diff
4,340332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,318182.0,318182.0,spatial join,same


In [5]:
# There should be no parcel with MAZ=999999 and HH >0
# if there are, need to manually fix them
print(fbp_2035_maz.loc[(fbp_2035_maz.maz_new==999999) & (fbp_2035_maz.tothh > 0)].shape[0])
print(fbp_2050_maz.loc[(fbp_2050_maz.maz_new==999999) & (fbp_2050_maz.tothh > 0)].shape[0])

1
0


In [6]:
# There should be no parcel with MAZ=999999 and emp >0
# if there are, need to manually fix them
print(fbp_2015_maz.loc[(fbp_2015_maz.maz_new==999999) & (fbp_2015_maz.totemp > 0)].shape[0])
print(fbp_2035_maz.loc[(fbp_2035_maz.maz_new==999999) & (fbp_2035_maz.totemp > 0)].shape[0])
print(fbp_2050_maz.loc[(fbp_2050_maz.maz_new==999999) & (fbp_2050_maz.totemp > 0)].shape[0])

10
12
12


In [7]:
# parcel with 2035 hh > 0
fbp_maz999999_nonzero35HH = fbp_2035_maz.loc[(fbp_2035_maz.maz_new==999999) & (fbp_2035_maz.tothh > 0)]
display(fbp_maz999999_nonzero35HH)

,PARCEL_ID,hhq1,hhq2,hhq3,hhq4,tothh,AGREMPN,MWTEMPN,RETEMPN,FPSEMPN,HEREMPN,OTHEMPN,totemp,maz_old,maz_new,maz_source,compare
1391848,628786,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999999.0,999999.0,oldLookup999999_zeroHH,same


In [8]:
# parcel with 2015 emp > 0
fbp_maz999999_nonzero15EMP = fbp_2015_maz.loc[(fbp_2015_maz.maz_new==999999) & (fbp_2015_maz.totemp > 0)]
display(fbp_maz999999_nonzero15EMP)

,PARCEL_ID,hhq1,hhq2,hhq3,hhq4,tothh,AGREMPN,MWTEMPN,RETEMPN,FPSEMPN,HEREMPN,OTHEMPN,totemp,maz_old,maz_new,maz_source,compare
1833,2054503,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,43.0,144.0,89.0,285.0,999999.0,999999.0,oldLookup,same
97732,782901,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,4.0,5.0,2.0,13.0,999999.0,999999.0,oldLookup999999_zeroHH,same
148610,782914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,1.0,2.0,7.0,999999.0,999999.0,oldLookup999999_zeroHH,same
163568,679589,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,4.0,2.0,8.0,999999.0,999999.0,oldLookup999999_zeroHH,same
636513,782907,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,4.0,7.0,4.0,16.0,999999.0,999999.0,oldLookup999999_zeroHH,same
737144,782913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,2.0,5.0,11.0,999999.0,999999.0,oldLookup999999_zeroHH,same
1069194,658441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,2.0,999999.0,999999.0,oldLookup999999_zeroHH,same
1069907,683850,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,3.0,3.0,9.0,999999.0,999999.0,oldLookup999999_zeroHH,same
1235998,782910,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4.0,4.0,4.0,14.0,999999.0,999999.0,oldLookup999999_zeroHH,same
1356353,750927,NaN,NaN,NaN,NaN,NaN,NaN,6.0,5.0,16.0,10.0,9.0,46.0,999999.0,999999.0,oldLookup999999_zeroHH,same


In [9]:
# parcel with 2035 emp > 0
fbp_maz999999_nonzero35EMP = fbp_2035_maz.loc[(fbp_2035_maz.maz_new==999999) & (fbp_2035_maz.totemp > 0)]
display(fbp_maz999999_nonzero35EMP)

,PARCEL_ID,hhq1,hhq2,hhq3,hhq4,tothh,AGREMPN,MWTEMPN,RETEMPN,FPSEMPN,HEREMPN,OTHEMPN,totemp,maz_old,maz_new,maz_source,compare
1833,2054503,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,68.0,176.0,76.0,326.0,999999.0,999999.0,oldLookup,same
97732,782901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.0,7.0,3.0,15.0,999999.0,999999.0,oldLookup999999_zeroHH,same
148610,782914,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,3.0,1.0,2.0,9.0,999999.0,999999.0,oldLookup999999_zeroHH,same
163568,679589,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,5.0,6.0,13.0,999999.0,999999.0,oldLookup999999_zeroHH,same
636513,782907,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,7.0,4.0,6.0,18.0,999999.0,999999.0,oldLookup999999_zeroHH,same
737144,782913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,6.0,5.0,14.0,999999.0,999999.0,oldLookup999999_zeroHH,same
1069194,658441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,3.0,5.0,10.0,999999.0,999999.0,oldLookup999999_zeroHH,same
1069907,683850,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,6.0,4.0,13.0,999999.0,999999.0,oldLookup999999_zeroHH,same
1161149,474877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,999999.0,999999.0,oldLookup999999_zeroHH,same
1235998,782910,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,5.0,3.0,3.0,12.0,999999.0,999999.0,oldLookup999999_zeroHH,same


In [10]:
# parcel with 2050 emp > 0
fbp_maz999999_nonzero50EMP = fbp_2050_maz.loc[(fbp_2050_maz.maz_new==999999) & (fbp_2050_maz.totemp > 0)]
display(fbp_maz999999_nonzero50EMP)

,PARCEL_ID,hhq1,hhq2,hhq3,hhq4,tothh,AGREMPN,MWTEMPN,RETEMPN,FPSEMPN,HEREMPN,OTHEMPN,totemp,maz_old,maz_new,maz_source,compare
1833,2054503,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,78.0,187.0,62.0,333.0,999999.0,999999.0,oldLookup,same
97732,782901,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,3.0,6.0,5.0,15.0,999999.0,999999.0,oldLookup999999_zeroHH,same
148610,782914,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,4.0,NaN,2.0,9.0,999999.0,999999.0,oldLookup999999_zeroHH,same
163568,679589,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,3.0,5.0,4.0,13.0,999999.0,999999.0,oldLookup999999_zeroHH,same
636513,782907,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,8.0,6.0,4.0,19.0,999999.0,999999.0,oldLookup999999_zeroHH,same
737144,782913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,4.0,14.0,999999.0,999999.0,oldLookup999999_zeroHH,same
1069194,658441,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,1.0,3.0,5.0,13.0,999999.0,999999.0,oldLookup999999_zeroHH,same
1069907,683850,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,7.0,3.0,13.0,999999.0,999999.0,oldLookup999999_zeroHH,same
1161149,474877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,999999.0,999999.0,oldLookup999999_zeroHH,same
1235998,782910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,3.0,13.0,999999.0,999999.0,oldLookup999999_zeroHH,same


In [11]:
# merge them, drop duplciates, and export
fbp_maz999999_nonzero35HH15EMP35EMP50EMP = pd.concat([fbp_maz999999_nonzero35HH,
                                                      fbp_maz999999_nonzero15EMP,
                                                      fbp_maz999999_nonzero35EMP,
                                                      fbp_maz999999_nonzero50EMP])
print(fbp_maz999999_nonzero35HH15EMP35EMP50EMP.shape[0])

# drop_duplicates and export
fbp_maz999999_nonzero35HH15EMP35EMP50EMP_nodup = fbp_maz999999_nonzero35HH15EMP35EMP50EMP[['PARCEL_ID', 'maz_new']].drop_duplicates()
print(fbp_maz999999_nonzero35HH15EMP35EMP50EMP_nodup.shape[0])
fbp_maz999999_nonzero35HH15EMP35EMP50EMP_nodup.to_csv(r'M:\Data\GIS layers\p10_TM2_maz\p10_mazfbp_maz999999_nonzero35HH15EMP35EMP50EMP.csv', index=False)

35
13


In [12]:
p_maz_lookup_new = p_maz_lookup.copy()

# manual fix 12 of the 13 parcels based on visual inspection in ArcGIS

p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 1236504, 'maz_new'] = 111913
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 1236504, 'maz_source'] = 'manual_assign'
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 1236504, 'compare'] = 'diff'

p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 750927, 'maz_new'] = 810302
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 750927, 'maz_source'] = 'manual_assign'
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 750927, 'compare'] = 'diff'

p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID.isin([782910, 782907, 782913, 782901]), 'maz_new'] = 813155
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID.isin([782910, 782907, 782913, 782901]), 'maz_source'] = 'manual_assign'
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID.isin([782910, 782907, 782913, 782901]), 'compare'] = 'diff'

p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 782914, 'maz_new'] = 814468
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 782914, 'maz_source'] = 'manual_assign'
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 782914, 'compare'] = 'diff'

p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID.isin([679589, 683850]), 'maz_new'] = 414760
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID.isin([679589, 683850]), 'maz_source'] = 'manual_assign'
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID.isin([679589, 683850]), 'compare'] = 'diff'

p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 474877, 'maz_new'] = 418791
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 474877, 'maz_source'] = 'manual_assign'
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 474877, 'compare'] = 'diff'

p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 658441, 'maz_new'] = 411123
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 658441, 'maz_source'] = 'manual_assign'
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 658441, 'compare'] = 'diff'

p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 628786, 'maz_new'] = 410032
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 628786, 'maz_source'] = 'manual_assign'
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 628786, 'compare'] = 'diff'


# one parcel (ID 2054503) does not have spatial info (will be removed if repairs shape in ArcGIS)
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 2054503, 'maz_new'] = 217456
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 2054503, 'maz_source'] = 'manual_assign'
p_maz_lookup_new.loc[p_maz_lookup_new.PARCEL_ID == 2054503, 'compare'] = 'diff'

In [13]:
# there are four synthetic parcels (2054504, 2054505, 2054506, 572927) in BAUS that do not have spatial info
# add all of them to MAZ 217456, TAZ 201085, Santa Clara

for parcel_id in [2054504, 2054505, 2054506, 572927]:
    p_maz_lookup_new = p_maz_lookup_new.append({'PARCEL_ID': parcel_id,
                                                'maz_new': 217456,
                                                'maz_source': 'manual_assign',
                                                'compare': 'diff'}, ignore_index=True)
    
# now the parcel-MAZ lookup should have 1956212 rows
print(p_maz_lookup_new.shape[0])

1956212


In [14]:
# export
display(p_maz_lookup_new.head())
p_maz_lookup_new.to_csv(r'M:\Data\GIS layers\p10_TM2_maz\p10_maz_lookup_compare_20220211.csv',
                        index=False)

,PARCEL_ID,maz_old,maz_new,maz_source,compare
0,229116,310596.0,324291.0,spatial join,diff
1,244166,331415.0,331415.0,spatial join,same
2,202378,310099.0,323229.0,spatial join,diff
3,2004420,710778.0,718260.0,spatial join,diff
4,340332,318182.0,318182.0,spatial join,same
